In [1]:
# General Imports
import sys
import os
import pandas as pd

# Modeling Imports
import pickle

# Import project modules
sys.path.append('../src/complete_collision_recorder/')
import complete_collision as cc
import utils as u

# Map project directories
root_dir, src_dir, data_dir, models_dir = u.map_project_directories(notebook=True)

/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(root_dir)

/media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder


In [3]:
training_df = pd.read_csv(os.path.join(data_dir, "processed", "training_df.csv"))
training_df['concatenated_text'] = training_df.apply(u.concatenate_texts, axis=1)

In [4]:
# Load the severity model
with open(os.path.join(models_dir,'severity.pkl'), 'rb') as f:
    severity_best = pickle.load(f)

In [5]:
training_df['SEVERITY_PRED'] = severity_best.predict(training_df['concatenated_text'])
training_df['SEVERITY_PRED'].value_counts()

SEVERITY_PRED
0    1923
1     252
Name: count, dtype: int64

In [6]:
GCAT_pred_df = training_df

In [7]:
GCAT_pred_df['SEVERITY_PRED_TEXT'] = GCAT_pred_df['SEVERITY_PRED'].replace({
    0: '',
    1: 'SEVERE INJURY',
})
# GCAT_pred_df['TURN_PRED_TEXT'] = GCAT_pred_df['TURN_PRED'].replace({
#     0: '',
#     1: 'TURN INVOLVED',
# })
GCAT_pred_df['GCAT_PRED_TEXT'] = GCAT_pred_df['SEVERITY_PRED_TEXT'] # + ' ' + GCAT_pred_df['TURN_PRED_TEXT']
GCAT_pred_df['GCAT_PRED_TEXT'] = GCAT_pred_df['GCAT_PRED_TEXT'].apply(lambda x: 'NONE' if x.strip() == '' else x)
GCAT_pred_df['concatenated_text'] = GCAT_pred_df['concatenated_text'] + '\n\n GCAT INDICATORS: ' + GCAT_pred_df['GCAT_PRED_TEXT']

In [8]:
GCAT_pred_df[['concatenated_text','BIKE_CLE_TEXT']].to_csv(os.path.join(data_dir, "processed", "GCAT_pred_df.csv"), index=False)